### Match ECHO videos with PET data (Rahul's lists) ###
We need this to assign the CFR labels to the videos. The goal is to have a data set with PET CFR values for each video that we will use to train.

In [2]:
import os
import pickle
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 2000)

In [3]:
cfr_data_root = os.path.normpath('/mnt/obi0/andreas/data/cfr')
cfr_meta_date = '200416'
meta_dir = os.path.join(cfr_data_root, 'metadata_'+cfr_meta_date)
print(meta_dir)

/mnt/obi0/andreas/data/cfr/metadata_200416


#### CFR REPORTS ####

In [3]:
pet_echo_file = os.path.normpath('/mnt/obi0/phi/pet/pet_cfr/pets_with_echos_global_all.parquet')
pet_echo_df = pd.read_parquet(pet_echo_file)

pet = pet_echo_df.rename(columns = {'pet_date': 'cfr_study_date'})
pet = pet.astype({'cfr_study_date': 'datetime64[ns]'})
#pet = pet.merge(right = rpet, on = ['mrn', 'cfr_study_date'], how='left')
print(pet.shape)
pet.head()

(3038, 52)


,mrn,cfr_study_date,petmrn_identifier,post-2018,myocardial_perfusion,global_cfr_calc,rest_global_mbf,stress_global_mbf,rest_seg1_mbf,rest_seg2_mbf,rest_seg3_mbf,rest_seg4_mbf,rest_seg5_mbf,rest_seg6_mbf,rest_seg7_mbf,rest_seg8_mbf,rest_seg9_mbf,rest_seg10_mbf,rest_seg11_mbf,rest_seg12_mbf,rest_seg13_mbf,rest_seg14_mbf,rest_seg15_mbf,rest_seg16_mbf,rest_seg17_mbf,stress_seg1_mbf,stress_seg2_mbf,stress_seg3_mbf,stress_seg4_mbf,stress_seg5_mbf,stress_seg6_mbf,stress_seg7_mbf,stress_seg8_mbf,stress_seg9_mbf,stress_seg10_mbf,stress_seg11_mbf,stress_seg12_mbf,stress_seg13_mbf,stress_seg14_mbf,stress_seg15_mbf,stress_seg16_mbf,stress_seg17_mbf,summed_stress_score,summed_rest_score,summed_difference_score,TID_ratio,gated_spect_results,agatston_coronary_calcium_score,height_in,weight_lb,reportID,subjectID
0,7924277,2008-08-15,7924277_2008-08-15,0,abnormal,1.368012,0.644,0.881,0.448,0.380,0.435,0.830,0.402,0.334,0.730,0.890,0.537,0.388,0.607,0.823,0.964,0.766,0.445,1.340,1.504,0.691,0.707,0.529,0.348,0.558,0.562,1.325,1.446,0.636,0.502,1.136,1.186,1.415,0.979,0.670,1.174,1.676,18.0,18.0,0.0,1.09,normal,NaN,67.0,133.0,69718.0,2540.0
1,12853099,2006-01-25,12853099_2006-01-25,0,normal,2.109661,1.532,3.232,1.082,0.940,1.135,1.224,1.332,1.945,1.482,1.977,1.475,1.656,1.509,1.444,1.801,2.180,1.677,1.359,1.977,4.291,3.144,2.069,2.657,2.626,3.712,3.981,3.359,2.215,4.010,3.890,3.883,3.249,2.609,3.483,3.443,3.037,0.0,0.0,0.0,None,not assessed,0.0,64.0,160.0,69969.0,3379.0
2,20710471,2006-01-23,20710471_2006-01-23,0,abnormal,1.229572,1.028,1.264,0.872,0.892,1.126,1.884,1.061,1.057,1.286,1.657,0.735,0.995,1.153,1.252,1.380,0.966,0.860,0.943,1.101,1.213,0.826,1.864,0.910,1.320,1.936,1.283,1.617,1.330,1.266,1.366,1.726,1.533,1.477,1.484,1.037,1.537,20.0,18.0,2.0,0.97,normal,6602.0,71.0,200.0,69993.0,3542.0
3,12627030,2006-02-10,12627030_2006-02-10,0,abnormal,1.588915,1.732,2.752,1.984,1.499,1.556,1.333,1.134,1.292,2.499,1.883,1.703,1.794,1.873,1.933,1.767,2.056,1.662,1.804,1.754,2.940,0.664,0.640,2.197,1.413,3.395,4.533,1.301,2.149,2.391,3.300,4.714,3.184,2.567,4.154,4.662,3.595,16.0,0.0,16.0,0.79,normal,331.0,68.0,217.0,70008.0,2155.0
4,15324312,2006-02-23,15324312_2006-02-23,0,normal,2.656655,1.465,3.892,0.945,0.813,0.892,0.970,1.401,1.217,1.614,1.580,1.303,1.957,1.879,1.794,1.421,1.773,1.677,1.891,1.846,2.559,2.759,2.654,2.434,3.313,3.502,4.081,4.340,3.978,4.170,4.426,4.944,4.343,4.143,4.748,4.544,4.328,0.0,0.0,0.0,0.94,normal,0.0,64.0,146.0,70012.0,3417.0


In [4]:
# Remove rows or columns with all NAs
pet2 = pet.dropna(how='all')

# Remove rows in pet df with missing values in very important columns
pet2 = pet.dropna(subset=['mrn', 'cfr_study_date', 'global_cfr_calc', 'rest_global_mbf'], axis=0)

print(pet2.shape)

(3038, 52)


In [5]:
# NEW PET_REPORT_ID: unique combination of mrn and study date
ID = pet2.groupby(['mrn', 'cfr_study_date']).ngroup()
pet2 = pet2.assign(mrnstudyid=ID)
print('Unique mrn_cfr_date_id values: {}'.format(len(pet2.mrnstudyid.unique())))
print('model_cfr values: {}'.format(len(pet2.global_cfr_calc.unique())))
print('Unique patients: {}'.format(len(pet2.mrn.unique())))

Unique mrn_cfr_date_id values: 3031
model_cfr values: 2985
Unique patients: 2703


#### NPY ECHO FILES ####
This is a data frame that contains the filenames, MRNs and dates of all echo studies

In [6]:
echo_files_name = 'echo_npyFiles_BWH_'+cfr_meta_date+'.parquet'
echo = pd.read_parquet(os.path.join(meta_dir, echo_files_name))
echo = echo.assign(echo_study_date = echo.datetime.dt.date,
                   echo_study_time = echo.datetime.dt.time)

echo = echo.astype({'mrn': 'int64',
                    'echo_study_date': 'datetime64[ns]'})

print(echo_files_name)
print('Number of npy files: {}'.format(len(echo.filename.unique())))
print('Unique echo studies: {}'.format(len(echo.study.unique())))
print('Unique patients:     {}'.format(len(echo.mrn.unique())))
print()

# For matching dates with PET, we only need the studies
# So we can greatly reduce this data frame
echo = echo[['study', 'mrn', 'echo_study_date']].drop_duplicates().reset_index(drop = True).\
    sort_values(by = 'mrn', ascending = True)

echo_npyFiles_BWH_200330.parquet
Number of npy files: 1296830
Unique echo studies: 25888
Unique patients:     15011



In [7]:
echo.head()

,study,mrn,echo_study_date
3728,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e,9241,2010-04-02
3723,4b7b463ae45ad0e0_4903a585842f436837ee2e0d9dd5,9241,2010-01-26
3809,4b7b463e0550e9de_4903a58051d59d426fe452739a29,41723,2015-11-03
3743,4b7b463fc2f49a36_4903a58793b43bb4bb1078d62f4e,51599,2012-03-08
3734,4b7b463fc2f49a36_4903a58dd5465407d53d20ba3790,51599,2018-05-22


#### PET STUDIES + NPY ECHO DATA ####

In [8]:
pet_echo = pet2.merge(right = echo, on = 'mrn', how = 'left')
pet_echo = pet_echo.sort_values(by='mrn')
pet_echo.iloc[5:20]

,mrn,cfr_study_date,petmrn_identifier,post-2018,myocardial_perfusion,global_cfr_calc,rest_global_mbf,stress_global_mbf,rest_seg1_mbf,rest_seg2_mbf,rest_seg3_mbf,rest_seg4_mbf,rest_seg5_mbf,rest_seg6_mbf,rest_seg7_mbf,rest_seg8_mbf,rest_seg9_mbf,rest_seg10_mbf,rest_seg11_mbf,rest_seg12_mbf,rest_seg13_mbf,rest_seg14_mbf,rest_seg15_mbf,rest_seg16_mbf,rest_seg17_mbf,stress_seg1_mbf,stress_seg2_mbf,stress_seg3_mbf,stress_seg4_mbf,stress_seg5_mbf,stress_seg6_mbf,stress_seg7_mbf,stress_seg8_mbf,stress_seg9_mbf,stress_seg10_mbf,stress_seg11_mbf,stress_seg12_mbf,stress_seg13_mbf,stress_seg14_mbf,stress_seg15_mbf,stress_seg16_mbf,stress_seg17_mbf,summed_stress_score,summed_rest_score,summed_difference_score,TID_ratio,gated_spect_results,agatston_coronary_calcium_score,height_in,weight_lb,reportID,subjectID,mrnstudyid,study,echo_study_date
3085,92742,2012-11-27,92742_2012-11-27,0,abnormal,1.338010,0.784,1.049,0.566,0.636,0.654,0.585,0.840,0.782,0.901,1.029,0.928,0.744,0.663,0.849,0.847,0.993,0.877,0.651,0.871,1.149,0.976,0.948,0.923,0.990,1.223,1.391,1.788,1.569,1.009,0.746,0.908,0.982,1.183,0.725,0.699,0.653,33.0,23.0,10.0,1.04,abnormal,NaN,64.0,175.0,120825.0,1646.0,5,4b7b4633cc96d8c2_4903a584a1befac2ae418ace04c3,2011-04-13
3086,92742,2012-11-27,92742_2012-11-27,0,abnormal,1.338010,0.784,1.049,0.566,0.636,0.654,0.585,0.840,0.782,0.901,1.029,0.928,0.744,0.663,0.849,0.847,0.993,0.877,0.651,0.871,1.149,0.976,0.948,0.923,0.990,1.223,1.391,1.788,1.569,1.009,0.746,0.908,0.982,1.183,0.725,0.699,0.653,33.0,23.0,10.0,1.04,abnormal,NaN,64.0,175.0,120825.0,1646.0,5,4b7b4633cc96d8c2_4903a585855f5450c473b5bd5d51,2010-11-26
2480,92742,2010-11-29,92742_2010-11-29,0,abnormal,1.097118,0.937,1.028,1.121,1.460,0.822,0.638,0.977,0.909,1.044,0.913,1.126,1.244,1.019,1.009,1.208,1.103,0.726,0.597,0.502,1.468,1.801,1.391,1.483,1.104,1.262,1.115,1.869,2.161,1.453,1.110,0.520,0.693,0.860,0.663,0.464,0.391,15.0,2.0,13.0,1.07,abnormal,NaN,66.0,164.0,107029.0,1646.0,4,4b7b4633cc96d8c2_4903a585855f5450c473b5bd5d51,2010-11-26
2479,92742,2010-11-29,92742_2010-11-29,0,abnormal,1.097118,0.937,1.028,1.121,1.460,0.822,0.638,0.977,0.909,1.044,0.913,1.126,1.244,1.019,1.009,1.208,1.103,0.726,0.597,0.502,1.468,1.801,1.391,1.483,1.104,1.262,1.115,1.869,2.161,1.453,1.110,0.520,0.693,0.860,0.663,0.464,0.391,15.0,2.0,13.0,1.07,abnormal,NaN,66.0,164.0,107029.0,1646.0,4,4b7b4633cc96d8c2_4903a584a1befac2ae418ace04c3,2011-04-13
2478,92742,2010-11-29,92742_2010-11-29,0,abnormal,1.097118,0.937,1.028,1.121,1.460,0.822,0.638,0.977,0.909,1.044,0.913,1.126,1.244,1.019,1.009,1.208,1.103,0.726,0.597,0.502,1.468,1.801,1.391,1.483,1.104,1.262,1.115,1.869,2.161,1.453,1.110,0.520,0.693,0.860,0.663,0.464,0.391,15.0,2.0,13.0,1.07,abnormal,NaN,66.0,164.0,107029.0,1646.0,4,4b7b4633cc96d8c2_4903a5858427dadde17abcd03be6,2010-09-27
3084,92742,2012-11-27,92742_2012-11-27,0,abnormal,1.338010,0.784,1.049,0.566,0.636,0.654,0.585,0.840,0.782,0.901,1.029,0.928,0.744,0.663,0.849,0.847,0.993,0.877,0.651,0.871,1.149,0.976,0.948,0.923,0.990,1.223,1.391,1.788,1.569,1.009,0.746,0.908,0.982,1.183,0.725,0.699,0.653,33.0,23.0,10.0,1.04,abnormal,NaN,64.0,175.0,120825.0,1646.0,5,4b7b4633cc96d8c2_4903a5858427dadde17abcd03be6,2010-09-27
1889,92742,2010-02-24,92742_2010-02-24,0,probably abnormal,1.336032,0.494,0.660,0.355,0.399,0.605,0.304,0.444,0.492,0.503,0.608,1.225,0.483,0.424,0.462,0.621,0.577,0.401,0.460,0.564,0.443,0.491,0.818,0.662,0.486,1.012,0.438,0.843,0.863,0.919,0.585,0.415,0.467,0.976,1.063,0.578,0.830,3.0,0.0,3.0,None,normal,NaN,66.0,200.0,101417.0,1646.0,3,4b7b4633cc96d8c2_4903a584a1befac2ae418ace04c3,2011-04-13
1890,92742,2010-02-24,92742_2010-02-24,0,probably abnormal,1.336032,0.494,0.660,0.355,0.399,0.605,0.304,0.444,0.492,0.503,0.608,1.225,0.483,0.424,0.462,0.621,0.577,0.401,0.460,0.564,0.443,0.491,0.818,0.662,0.486,1.012,0.438,0.843,0.863,0.919,0.585,0.415,0.467,0.976,1.063,0.578,0.830,3.0,0.0,3.0,None,normal,NaN,66.0,200.0,101417.0,1646.0,3,4b7b4633cc96d8c2_4903a585855f5450c473b5bd5d51,2010-11-26
13

In [9]:
# Filter NAN in study from merged PET report (we don't have .npy files from those studies)
pet_echo_nofiles = pet_echo.loc[pet_echo.study.isnull()].reset_index(drop = True)
petecho = pet_echo.loc[~pet_echo.study.isnull()].reset_index(drop = True)

In [10]:
print('PET with echo videos:')
#print('Number of npy files: {}'.format(len(petecho.filename.unique())))
print('Unique echo studies: {}'.format(len(petecho.study.unique())))
print('Unique patients:     {}'.format(len(petecho.mrn.unique())))

PET with echo videos:
Unique echo studies: 3851
Unique patients:     2270


In [11]:
print('PET without ECHO:')
#print('Number of npy files: {}'.format(len(petecho.filename.unique())))
print('Unique CFR studies: {}'.format(len(pet_echo_nofiles.reportID.unique())))
print('Unique patients:     {}'.format(len(pet_echo_nofiles.mrn.unique())))

PET without ECHO:
Unique CFR studies: 465
Unique patients:     433


### Match ECHO study with PET reportID  ###
Goal:
Find a CFR label for each echo video taken from the closest PET measurement.
Requirement: We want to pick the most recent CFR measurments for each echo that is not older than 365 days.
For this, we need a data frame that contains for a given MRN:

- All echo studies for this patient
- A date for each echo study
- All pet reports for that patient
- Date for each pet report

In [12]:
def get_earliest_cfr_reports(df_cfr_echo):
    """ 
    Determine the earliest CFR measurement for all studies
    args: df_cfr_echo: pd.DataFrame() 
          with columns: mrn, study, reportID, cfr_study_date, echo_study_date
    returns: cfr_echo_days: pd.DataFrame()
          with columns: mrn, study, reportID, cfr_days_after_echo
    """

    # Loop through all MRNs in df_pet_echo
    mrn_list = sorted(list(df_cfr_echo.mrn.unique()))
    cfr_echo_days = pd.DataFrame()
    for m, mrn in enumerate(mrn_list):

        if (m+1)%200==0:
            print('Processing MRN {} of {}'.format(m+1, len(mrn_list)))

        df_mrn = df_cfr_echo[df_cfr_echo.mrn == mrn]
        echo_study_list = list(df_mrn.study.unique())

        # Loop through all echo studies for this MRN
        for study in echo_study_list:
            echo_study_date = df_mrn[df_mrn.study == study].echo_study_date.values[0]

            # Find all pet dates for this patient and add the difference to the echo dates
            pet_dates = df_mrn[['mrnstudyid', 'cfr_study_date']].groupby('mrnstudyid').first().reset_index(drop=False)
            pet_dates = pet_dates.assign(days_post_cfr=(echo_study_date-pet_dates.cfr_study_date).dt.days)
            pet_dates = pet_dates.assign(days_abs = np.abs(pet_dates.days_post_cfr))
            ser = pet_dates.loc[pet_dates.days_abs.idxmin()]

            # We can return the reportID and the days of the earliest pet study
            study_dict = {'mrn': [mrn],
                          'study': [study],
                          'echo_study_date': [echo_study_date],
                          'mrnstudyid': [ser.mrnstudyid],
                          'days_post_cfr': [ser.days_post_cfr]}
            cfr_echo_days = pd.concat([cfr_echo_days, pd.DataFrame(study_dict)], ignore_index=True)
    
    return cfr_echo_days

In [13]:
cfr_echo_days = get_earliest_cfr_reports(petecho)

Processing MRN 200 of 2270
Processing MRN 400 of 2270
Processing MRN 600 of 2270
Processing MRN 800 of 2270
Processing MRN 1000 of 2270
Processing MRN 1200 of 2270
Processing MRN 1400 of 2270
Processing MRN 1600 of 2270
Processing MRN 1800 of 2270
Processing MRN 2000 of 2270
Processing MRN 2200 of 2270


In [14]:
# After the match, do we have the same number of patients
print('Patients after match     {}'.format(len(cfr_echo_days.mrn.unique())))
print('Echo studies after match {}'.format(len(cfr_echo_days.study.unique())))

Patients after match     2270
Echo studies after match 3851


In [15]:
# We can merge this back with the original PET reports
pet_cfr_echo_days = cfr_echo_days.merge(right = pet2, on = ['mrnstudyid', 'mrn'], how = 'left')

In [16]:
# After the match, do we have the same number of patients
print('Patients after match     {}'.format(len(pet_cfr_echo_days.mrn.unique())))
print('Echo studies after match {}'.format(len(pet_cfr_echo_days.study.unique())))

Patients after match     2270
Echo studies after match 3851


In [17]:
# Next, we will filter those echo studies that occurred earlier or later than 365 days after CFR
pet_cfr_echo_days_365 = pet_cfr_echo_days.assign(abs_cfr_days = np.abs(pet_cfr_echo_days.days_post_cfr))
pet_cfr_echo_days_365 = pet_cfr_echo_days_365[pet_cfr_echo_days_365.abs_cfr_days<=365].drop(columns = ['abs_cfr_days'])
# Sort this by study so that we know how well we matched the dates
pet_cfr_echo_days_365 = pet_cfr_echo_days_365.sort_values(by = 'study')

In [18]:
# After the match, do we have the same number of patients
print('Patients after 365 filter     {}'.format(len(pet_cfr_echo_days_365.mrn.unique())))
print('Echo studies after 365 filter {}'.format(len(pet_cfr_echo_days_365.study.unique())))

Patients after 365 filter     2163
Echo studies after 365 filter 3067


In [19]:
# Patients who dropped out because of the 365 time limit
mrn_before_filter = set(list(cfr_echo_days.mrn.unique()))
mrn_after_filter = set(list(pet_cfr_echo_days_365.mrn.unique()))
mrn_365_dropped = list(mrn_before_filter.difference(mrn_after_filter))
print('MRNs lost after 365 filter {}'.format(len(mrn_365_dropped)))
mrn_dropped_file = os.path.join(meta_dir, 'mrn_365_dropped.pkl')
with open(mrn_dropped_file, 'wb') as fl:
    pickle.dump(mrn_365_dropped, fl)

MRNs lost after 365 filter 107


In [20]:
# To match with filenames, we really only need one study per row. 
# Remove colums where some rows have missing values
pet_cfr_echo_days_noNA = pet_cfr_echo_days_365.dropna(axis=1, how='any') 
pet_cfr_echo_days_study = pet_cfr_echo_days_noNA.groupby('study').first().reset_index(drop=False)

print('Original shape: {} now: {}'.format(cfr_echo_days.shape,
                                          pet_cfr_echo_days_365.shape))

print('Original number of studies: {} now: {}'.format(len(cfr_echo_days.study.unique()),
                                                      len(pet_cfr_echo_days_study.study.unique())))

Original shape: (3851, 5) now: (3075, 56)
Original number of studies: 3851 now: 3067


In [21]:
d = pet_cfr_echo_days_study[['mrn', 'study', 'cfr_study_date', 'echo_study_date', 'days_post_cfr']]
d.head(10)

,mrn,study,cfr_study_date,echo_study_date,days_post_cfr
0,35169325,48b09013185d7d6b_4903a582ec77f16c8e735c3b6113,2017-05-22,2017-06-21,30
1,35133560,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,2018-04-25,2018-11-02,191
2,35014216,48b091b44637c987_4903a58c1bcb23767fc2818c6e63,2018-05-08,2019-04-25,352
3,35034941,48b091b6184a771a_4903a582ec751999e07cf30ae81a,2017-04-17,2017-04-18,1
4,35029545,48b091b714b41ffe_4903a582ec746dce5e9b6c1f4070,2017-07-24,2017-05-16,-69
5,35092402,48b091bc02fb15f5_4903a58dd541f0af3fb9763595d6,2018-01-29,2018-02-12,14
6,35583509,48b09412e8754782_4903a582ec78ec0366e9db9e17cd,2018-08-08,2017-09-06,-336
7,35401116,48b095533cbfdf01_4903a58c1bcb2198682a98b7503a,2019-04-11,2019-04-09,-2
8,35401116,48b095533cbfdf01_4903a58dd5441181fcd696f7d50c,2019-04-11,2018-07-25,-260
9,35750744,48b09627edb790dc_4903a582ec78ec0420a169da26a4,2017-08-16,2017-09-01,16


In [22]:
pet_cfr_echo_days_study.head(2)

,study,mrn,echo_study_date,mrnstudyid,days_post_cfr,cfr_study_date,petmrn_identifier,post-2018,global_cfr_calc,rest_global_mbf,stress_global_mbf
0,48b09013185d7d6b_4903a582ec77f16c8e735c3b6113,35169325,2017-06-21,2971,30,2017-05-22,35169325_2017-05-22,0,1.329310,0.580,0.771
1,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02,2969,191,2018-04-25,35133560_2018-04-25,0,1.716143,0.923,1.584


In [23]:
print(pet_cfr_echo_days_study.shape)
pet_cfr_echo_days_study.head()

(3067, 11)


,study,mrn,echo_study_date,mrnstudyid,days_post_cfr,cfr_study_date,petmrn_identifier,post-2018,global_cfr_calc,rest_global_mbf,stress_global_mbf
0,48b09013185d7d6b_4903a582ec77f16c8e735c3b6113,35169325,2017-06-21,2971,30,2017-05-22,35169325_2017-05-22,0,1.329310,0.580,0.771
1,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02,2969,191,2018-04-25,35133560_2018-04-25,0,1.716143,0.923,1.584
2,48b091b44637c987_4903a58c1bcb23767fc2818c6e63,35014216,2019-04-25,2965,352,2018-05-08,35014216_2018-05-08,0,2.644905,0.628,1.661
3,48b091b6184a771a_4903a582ec751999e07cf30ae81a,35034941,2017-04-18,2967,1,2017-04-17,35034941_2017-04-17,0,1.875930,0.403,0.756
4,48b091b714b41ffe_4903a582ec746dce5e9b6c1f4070,35029545,2017-05-16,2966,-69,2017-07-24,35029545_2017-07-24,0,2.698341,0.663,1.789


In [24]:
print('Number of echo studies within 365 days of cfr {}'.format(len(pet_cfr_echo_days_study.study.unique())))
print('Number of PET studies {}'.format(len(pet_cfr_echo_days_study.mrnstudyid.unique())))
print('Total number of studies in petecho df {}'.format(len(petecho.study.unique())))
print('Unique patients:     {}'.format(len(pet_cfr_echo_days_study.mrn.unique())))

Number of echo studies within 365 days of cfr 3067
Number of PET studies 2239
Total number of studies in petecho df 3851
Unique patients:     2163


In [25]:
# We can now save the data frame with the echo studies and the matched CFR measurements
match_filename = 'pet_match365_diff_'+cfr_meta_date+'.parquet'
pet_cfr_echo_days_study.to_parquet(os.path.join(meta_dir, match_filename))

# Save also as .csv file
match_filename_csv = 'pet_match365_diff_'+cfr_meta_date+'.csv'
pet_cfr_echo_days_study.to_csv(os.path.join(meta_dir, match_filename_csv))

In [26]:
pet_cfr_echo_days_study.head()

,study,mrn,echo_study_date,mrnstudyid,days_post_cfr,cfr_study_date,petmrn_identifier,post-2018,global_cfr_calc,rest_global_mbf,stress_global_mbf
0,48b09013185d7d6b_4903a582ec77f16c8e735c3b6113,35169325,2017-06-21,2971,30,2017-05-22,35169325_2017-05-22,0,1.329310,0.580,0.771
1,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02,2969,191,2018-04-25,35133560_2018-04-25,0,1.716143,0.923,1.584
2,48b091b44637c987_4903a58c1bcb23767fc2818c6e63,35014216,2019-04-25,2965,352,2018-05-08,35014216_2018-05-08,0,2.644905,0.628,1.661
3,48b091b6184a771a_4903a582ec751999e07cf30ae81a,35034941,2017-04-18,2967,1,2017-04-17,35034941_2017-04-17,0,1.875930,0.403,0.756
4,48b091b714b41ffe_4903a582ec746dce5e9b6c1f4070,35029545,2017-05-16,2966,-69,2017-07-24,35029545_2017-07-24,0,2.698341,0.663,1.789


In [27]:
len(pet_cfr_echo_days_study.mrn.unique())

2163

#### Add Rahul's match list ####

In [4]:
match_list_file = 'bwh_2020-Feb-4_pet_365_diff.txt'
rdf = pd.read_csv(os.path.join(cfr_data_root, match_list_file), sep = '\t')
rdf = rdf.astype({'echo_date': 'datetime64[ns]',
                  'pet_date': 'datetime64[ns]'})

In [5]:
rdf.head()

,MRN,echo_date,pet_date,difference(days)
0,17666439,2017-03-14,2017-07-20,128
1,12287447,2019-05-16,2020-01-23,252
2,12287447,2020-01-23,2020-01-23,0
3,31953938,2017-11-09,2018-08-20,284
4,31953938,2018-08-13,2018-08-20,7


In [28]:
match_list_file = 'bwh_2020-Feb-4_pet_365_diff.txt'
rdf = pd.read_csv(os.path.join(cfr_data_root, match_list_file), sep = '\t')
rdf = rdf.astype({'echo_date': 'datetime64[ns]',
                  'pet_date': 'datetime64[ns]'})

pet_cfr = pet_cfr_echo_days.rename(columns = {'cfr_study_date': 'pet_date',
                                              'echo_study_date': 'echo_date',
                                              'mrn': 'MRN',
                                              'days_post_cfr': 'days_post_pet'}).sort_values(by='MRN')


pet_cols = ['MRN', 'study', 'pet_date', 'echo_date', 'mrnstudyid', 'petmrn_identifier', 'days_post_pet']

pet_cfr = pet_cfr[pet_cols].dropna(axis=1, how='any') 
pet_cfr = pet_cfr.groupby('study').first().reset_index(drop=False).sort_values(by='MRN')

In [29]:
rdf_match = rdf.merge(right=pet_cfr, on=['MRN', 'echo_date', 'pet_date'], how='left').sort_values(by='MRN')
rdf_match.head(50)

,MRN,echo_date,pet_date,difference(days),study,mrnstudyid,petmrn_identifier,days_post_pet
2562,9241,2010-01-26,2010-01-28,2,4b7b463ae45ad0e0_4903a585842f436837ee2e0d9dd5,0.0,9241_2010-01-28,-2.0
2563,9241,2010-04-02,2010-01-28,64,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e,0.0,9241_2010-01-28,64.0
564,59691,2019-11-18,2019-01-17,305,NaN,NaN,NaN,NaN
566,59691,2016-09-22,2016-09-22,0,NaN,NaN,NaN,NaN
565,59691,2015-08-31,2015-09-17,17,NaN,NaN,NaN,NaN
563,59691,2018-11-19,2019-01-17,59,NaN,NaN,NaN,NaN
2564,68189,2013-10-22,2013-10-22,0,NaN,NaN,NaN,NaN
2565,81182,2008-01-04,2007-08-17,140,NaN,NaN,NaN,NaN
2566,81182,2007-08-13,2007-08-17,4,NaN,NaN,NaN,NaN
2567,81182,2007-08-14,2007-08-17,3,NaN,NaN,NaN,NaN


In [85]:
print('Total pet-echo pairs:           {}'.format(rdf_match.shape[0]))
print('Total unique MRNs:              {}'.format(len(rdf_match.MRN.unique())))
rdf_match_study = rdf_match.loc[~rdf_match.study.isnull()]
rdf_match_no_study = rdf_match.loc[rdf_match.study.isnull()].dropna(axis=1, how='all')
print()
print('Pet-echo pairs w/echo on FS:    {}'.format(rdf_match_study.shape[0]))
print('Pet-echo pairs without echo:    {}'.format(rdf_match_no_study.shape[0]))

print('MRNs with with echo on FS  :    {}'.format(len(rdf_match_study.MRN.unique())))
print('MRNs with missing echos    :    {}'.format(len(rdf_match_no_study.MRN.unique())))

Total pet-echo pairs:           7425
Total unique MRNs:              3567

Pet-echo pairs w/echo on FS:    2920
Pet-echo pairs without echo:    4505
MRNs with with echo on FS  :    2124
MRNs with missing echos    :    2122


In [86]:
study_mrn = set(rdf_match_study.MRN.unique())
nostudy_mrn = set(rdf_match_no_study.MRN.unique())
inters = study_mrn.intersection(nostudy_mrn)
# There are multiple PETs per patient. Therefore, some patients with echos are missing echos for other pet scans
rdf_match[rdf_match.MRN==1210558]

,MRN,echo_date,pet_date,difference(days),study,mrnstudyid,petmrn_identifier,days_post_pet
2716,1210558,2007-10-31,2008-03-05,126,4b7a857af794b692_4903a444c23a29accd772d454581,88.0,1210558_2008-03-05,-126.0
2711,1210558,2009-02-19,2008-03-05,351,NaN,NaN,NaN,NaN
2712,1210558,2009-02-19,2009-06-17,118,NaN,NaN,NaN,NaN
2713,1210558,2009-02-26,2008-03-05,358,NaN,NaN,NaN,NaN
2715,1210558,2007-06-23,2008-03-05,256,NaN,NaN,NaN,NaN
2714,1210558,2009-02-26,2009-06-17,111,4b7a857af794b692_4903a44ab129ec130a69ee54d70b,89.0,1210558_2009-06-17,-111.0


In [51]:
rdf_match_no_study

,MRN,echo_date,pet_date,difference(days)
564,59691,2019-11-18,2019-01-17,305
566,59691,2016-09-22,2016-09-22,0
565,59691,2015-08-31,2015-09-17,17
563,59691,2018-11-19,2019-01-17,59
2564,68189,2013-10-22,2013-10-22,0
...,...,...,...,...
63,39996913,2019-07-15,2019-12-19,157
163,40316176,2019-10-07,2019-10-09,2
155,40611006,2019-12-04,2019-10-17,48
156,40611006,2019-09-28,2019-10-17,19


In [87]:
rdf_match_no_study.dtypes

MRN                          int64
echo_date           datetime64[ns]
pet_date            datetime64[ns]
difference(days)             int64
dtype: object

In [88]:
# For those missing studies, we might have .dcm files. Lets take a look
dcm_df_file = 'echo_deIdentifyedEcho_BWH_dcm_200330.parquet'
dcm_df = pd.read_parquet(os.path.join(meta_dir, dcm_df_file))
# Just get the studies
dcm_df_study = dcm_df.groupby('study').first().reset_index(drop=False).drop(columns='filename')
dcm_df_study.datetime = dcm_df_study.datetime.dt.date
dcm_df_study = dcm_df_study.astype({'datetime': 'datetime64[ns]',
                                    'mrn': 'int64'})
dcm_df_study = dcm_df_study.rename(columns={'mrn': 'MRN',
                                            'datetime': 'echo_date'})

# Try to merge with missing pet-echo-pairs
rdf_match_no_study_dcm = rdf_match_no_study.merge(right=dcm_df_study, on=['MRN', 'echo_date'], how='left')

In [89]:
rdf_match_no_study_withdcm = rdf_match_no_study_dcm.loc[~rdf_match_no_study_dcm.study.isnull()]
rdf_match_no_study_withoutdcm = rdf_match_no_study_dcm[rdf_match_no_study_dcm.study.isnull()]
print('Pet_echo pairs w/o  .npy data on FS: {}'.format(rdf_match_no_study.shape[0]))
print('Pet-echo pairs with .dcm data on FS: {}'.format(rdf_match_no_study_withdcm.shape[0]))
print('Pet-echo pairs w/o  .dcm data on FS: {}'.format(rdf_match_no_study_withoutdcm.shape[0]))
# Save the list of echos with and without dcm data
rdf_match_no_study_dcm_file = 'bwh_2020-Feb-4_pet_365_missing_dcm.parquet'
rdf_match_no_study_dcm.to_parquet(os.path.join(meta_dir, rdf_match_no_study_dcm_file))

Pet_echo pairs w/o  .npy data on FS: 4505
Pet-echo pairs with .dcm data on FS: 2985
Pet-echo pairs w/o  .dcm data on FS: 1520


In [90]:
len(rdf_match_no_study_withoutdcm.MRN.unique())

931

In [93]:
rdf_match_no_study_dcm.shape

(4505, 6)

In [94]:
df = pd.read_parquet(os.path.join(meta_dir, rdf_match_no_study_dcm_file))